In [ ]:
# Reference: https://pypi.org/project/diffusers/

#### Setup

In [ ]:
!pip install diffusers[torch] transformers accelerate

In [ ]:
!apt-get install git-lfs

In [ ]:
%mkdir -p /pt/diffusers/stable-diffusion-v1-5
%cd /pt/diffusers/stable-diffusion-v1-5

!git lfs install

%cd /pt/diffusers/
#!git lfs

In [ ]:
!git clone https://huggingface.co/runwayml/stable-diffusion-v1-5

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline

device = "cuda"
model_id = "runwayml/stable-diffusion-v1-5"

# Safety Checker turned off to free GPU memory - https://github.com/CompVis/stable-diffusion/issues/86
# Memory management options - using 16-bit floats
safety_checker = None
torch_dtype = torch.float16
variant = "fp16"

In [ ]:
stable_diffusion_txt2img = StableDiffusionPipeline.from_pretrained(model_id, variant=variant, torch_dtype=torch_dtype, safety_checker=safety_checker)

components = stable_diffusion_txt2img.components

# weights are not reloaded into RAM - https://huggingface.co/docs/diffusers/using-diffusers/loading#loading-customized-pipelines
stable_diffusion_img2img = StableDiffusionImg2ImgPipeline(**components)

#### Text-to-Image generation with Stable Diffusion

In [ ]:
stable_diffusion_txt2img = stable_diffusion_txt2img.to(device)

prompt = "a photo of an astronaut riding a horse on mars"
image = stable_diffusion_txt2img(prompt).images[0]

from IPython import display
display.display(image)

In [ ]:
# Optionally save output
image.save("astronaut_rides_horse.png")

 #### Image-to-Image text-guided generation with Stable Diffusion

In [ ]:
import os.path
url, filename = ("https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg", "./input/sketch-mountains-input.jpg")

if os.path.isfile(filename):
    print("Image file exists")
else:
    import urllib.request
    urllib.request.urlretrieve(url, filename)
    print("Image file downloaded")
    

In [ ]:
from PIL import Image

# Memory management options = image size set to 512, 512
image_size = (512, 512)
filename = "./input/sketch-mountains-input.jpg"

init_image = Image.open(filename).convert("RGB")
init_image = init_image.resize(image_size)
#display.display(init_image)

prompt =  "A fantasy landscape, trending on artstation"

stable_diffusion_img2img = stable_diffusion_img2img.to(device)

In [ ]:
generator = torch.Generator(device=device).manual_seed(1024)
images = stable_diffusion_img2img(prompt=prompt, image=init_image, strength=0.75, guidance_scale=7.5, generator=generator, num_inference_steps=100).images

from IPython import display
display.display(images[0])

In [ ]:
stable_diffusion_img2img.save_pretrained("./stable-diffusion-v1-5", variant=variant)